<a href="https://colab.research.google.com/github/LEECHANGDAEE/datacon/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%ACnew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, get_scorer
from sklearn.linear_model import Lasso, Ridge, LassoCV,LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, RepeatedKFold, GridSearchCV, cross_validate, train_test_split

In [77]:

train = pd.read_csv('./train2.csv',encoding='cp949')
test = pd.read_csv('./test1.csv',encoding='cp949')
submission = pd.read_csv('./sample_submission1.csv',encoding='cp949')

train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

train= pd.get_dummies(data = train, columns = ['임대건물구분'], prefix = '건물구분')
test =  pd.get_dummies(data = test, columns = ['임대건물구분'], prefix = '건물구분')


In [78]:
data_m = train.groupby('단지코드').sum()
data_mt = test.groupby('단지코드').sum()

len(data_m)
data_st = data_m.건물구분_상가
data_ap = data_m.건물구분_아파트

data_stt = data_mt.건물구분_상가
data_apt = data_mt.건물구분_아파트

In [79]:
train['전용면적'] = round(train['전용면적'])//5*5
test['전용면적'] = round(test['전용면적'])//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 1
idx = train[train['전용면적']>50].index
train.loc[idx, '전용면적'] = 2
idx = train[train['전용면적']>40].index
train.loc[idx, '전용면적'] = 3
idx = train[train['전용면적']>9].index
train.loc[idx, '전용면적'] = 4


idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 1
idx = test[test['전용면적']>50].index
test.loc[idx, '전용면적'] = 2
idx = test[test['전용면적']>40].index
test.loc[idx, '전용면적'] = 3
idx = test[test['전용면적']>9].index
test.loc[idx, '전용면적'] = 4

columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []

for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

new_train = pd.DataFrame()
new_test = pd.DataFrame()

for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
  
new_train = new_train.fillna(0)
new_test = new_test.fillna(0)


411it [00:04, 98.44it/s]
150it [00:01, 106.47it/s]


In [80]:

nw = new_train.set_index('단지코드')
ntw = new_test.set_index('단지코드')


In [81]:
d1 = pd.concat([nw.iloc[:,:-1],data_ap],axis=1)
d2 = pd.concat([d1,data_st],axis=1)
d2 = d2.reset_index()

dt1 = pd.concat([ntw,data_apt],axis=1)
dt2 = pd.concat([dt1,data_stt],axis=1)
dt2 = dt2.reset_index()



In [82]:

x = d2.iloc[:,1:]
y = new_train['등록차량수']

In [83]:
yy = dt2.iloc[:,1:]
yy.head()

,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_4.0,면적_3.0,면적_2.0,면적_1.0,건물구분_아파트,건물구분_상가
0,754.0,14.0,경기도,683.0,0.0,2.0,116.0,638.0,0.0,0.0,8,0
1,1354.0,9.0,경기도,1216.0,0.0,3.0,420.0,934.0,0.0,0.0,9,0
2,619.0,18.0,부산광역시,547.0,0.0,16.0,247.0,328.0,44.0,0.0,9,0
3,593.0,7.0,전라북도,543.0,0.0,3.0,335.0,258.0,0.0,0.0,4,0
4,1297.0,11.0,경기도,1112.0,0.0,2.0,425.0,872.0,0.0,0.0,8,0


In [71]:
new_train

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_4.0,면적_3.0,면적_2.0,면적_1.0,등록차량수
0,C2515,545.0,17.0,경상남도,624.0,0.0,3.0,356.0,189.0,0.0,0.0,205.0
1,C1407,1216.0,13.0,대전광역시,1285.0,1.0,1.0,752.0,464.0,0.0,0.0,1064.0
2,C1945,755.0,6.0,경기도,734.0,1.0,3.0,240.0,303.0,212.0,0.0,730.0
3,C1470,696.0,14.0,전라북도,645.0,0.0,2.0,254.0,442.0,0.0,0.0,553.0
4,C1898,566.0,9.0,전라북도,517.0,0.0,6.0,271.0,295.0,0.0,0.0,415.0
...,...,...,...,...,...,...,...,...,...,...,...,...
406,C2586,90.0,7.0,제주특별자치도,66.0,0.0,3.0,90.0,0.0,0.0,0.0,57.0
407,C2035,492.0,24.0,강원도,521.0,0.0,1.0,312.0,180.0,0.0,0.0,246.0
408,C2020,40.0,7.0,부산광역시,25.0,1.0,2.0,40.0,0.0,0.0,0.0,19.0
409,C2437,90.0,12.0,충청북도,30.0,0.0,1.0,90.0,0.0,0.0,0.0,16.0


In [72]:
for i in range(len(x)) :
  if x.지역[i] in ['광주광역시','부산광역시','대전광역시','울산광역시','대구광역시'] :
    x.지역[i] = '광역시'
  elif x.지역[i] in ['서울특별시','세종특별자치시','제주특별자치도'] :
    x.지역[i] = '특별자치시'
  elif x.지역[i] in ['경상남도','경상북도','충청북도','충청남도','전라남도','전라북도'] :
    x.지역[i] = '지방시'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [84]:
x = pd.get_dummies(data = x, columns = ['지역'], prefix = '지역')



In [85]:
x

,총세대수,공가수,단지내주차면수,지하철,버스,면적_4.0,면적_3.0,면적_2.0,면적_1.0,건물구분_아파트,건물구분_상가,지역_강원도,지역_경기도,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,지역_부산광역시,지역_서울특별시,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도
0,545.0,17.0,624.0,0.0,3.0,356.0,189.0,0.0,0.0,7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1216.0,13.0,1285.0,1.0,1.0,752.0,464.0,0.0,0.0,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,755.0,6.0,734.0,1.0,3.0,240.0,303.0,212.0,0.0,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,696.0,14.0,645.0,0.0,2.0,254.0,442.0,0.0,0.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,566.0,9.0,517.0,0.0,6.0,271.0,295.0,0.0,0.0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,90.0,7.0,66.0,0.0,3.0,90.0,0.0,0.0,0.0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
407,492.0,24.0,521.0,0.0,1.0,312.0,180.0,0.0,0.0,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
408,40.0,7.0,25.0,1.0,2.0,40.0,0.0,0.0,0.0,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
409,90.0,12.0,30.0,0.0,1.0,90.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
for i in range(50) :
  
  x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2,random_state =i)
  mlr = LinearRegression()
  mlr.fit(x_train, y_train)

  ridge = Ridge(alpha = 0.5).fit(x_train, y_train)

  lasso = Lasso(alpha=0.2).fit(x_train, y_train)

  y_linear_predict = mlr.predict(x_test)
  y_ridge_predict = ridge.predict(x_test)
  y_lasso_predict = lasso.predict(x_test)
  from sklearn.metrics import mean_absolute_error
  from sklearn.metrics import mean_squared_error
  print(mean_absolute_error(y_lasso_predict,y_test))

In [87]:

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2,random_state =0)
mlr = LinearRegression()
mlr.fit(x_train, y_train)

ridge = Ridge(alpha = 0.5).fit(x_train, y_train)

lasso = Lasso(alpha=0.2).fit(x_train, y_train)

y_linear_predict = mlr.predict(x_test)
y_ridge_predict = ridge.predict(x_test)
y_lasso_predict = lasso.predict(x_test)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print('linear regression rmse : {}'.format(mean_squared_error(y_linear_predict,y_test)**0.5))
print('ridge regression rmse : {}'.format(mean_squared_error(y_ridge_predict,y_test)**0.5))
print('lasso regression rmse : {}'.format(mean_squared_error(y_lasso_predict,y_test)**0.5))
print(mean_absolute_error(y_lasso_predict,y_test))

linear regression rmse : 131.89408962127544
ridge regression rmse : 131.3990300003483
lasso regression rmse : 131.44183394250467
94.68897901610161


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3398276.058007823, tolerance: 4977.526067987805
  positive)


In [21]:

for i in range(len(yy)) :
  if yy.지역[i] in ['광주광역시','부산광역시','대전광역시','울산광역시','대구광역시'] :
    yy.지역[i] = '광역시'
  elif yy.지역[i] in ['서울특별시','세종특별자치시','제주특별자치도'] :
    yy.지역[i] = '특별자치시'
  elif yy.지역[i] in ['경상남도','경상북도','충청북도','충청남도','전라남도','전라북도'] :
    yy.지역[i] = '지방시'
yy = pd.get_dummies(data = yy, columns = ['지역'], prefix = '지역')
yy


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,총세대수,공가수,단지내주차면수,지하철,버스,면적_30.0,면적_40.0,면적_45.0,면적_50.0,면적_60.0,면적_35.0,면적_25.0,면적_70.0,면적_10.0,면적_15.0,면적_20.0,면적_55.0,면적_120.0,면적_75.0,면적_80.0,면적_85.0,면적_65.0,건물구분_아파트,건물구분_상가,지역_강원도,지역_경기도,지역_광역시,지역_지방시,지역_특별자치시
0,754.0,14.0,683.0,0.0,2.0,0.0,116.0,262.0,376.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0,0,1,0,0,0
1,1354.0,9.0,1216.0,0.0,3.0,0.0,420.0,356.0,578.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,0,0,1,0,0,0
2,619.0,18.0,547.0,0.0,16.0,82.0,165.0,196.0,132.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,0,0,0,1,0,0
3,593.0,7.0,543.0,0.0,3.0,0.0,335.0,174.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0,0,1,0
4,1297.0,11.0,1112.0,0.0,2.0,0.0,425.0,568.0,304.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,349.0,17.0,270.0,0.0,4.0,0.0,0.0,96.0,0.0,0.0,138.0,112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0,0,0,1
146,596.0,35.0,593.0,0.0,1.0,0.0,0.0,107.0,0.0,0.0,304.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,0,0,0,1,0
147,120.0,9.0,40.0,0.0,1.0,54.0,0.0,0.0,0.0,0.0,0.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,1,0,0,0,0
148,675.0,38.0,467.0,0.0,1.0,0.0,0.0,180.0,0.0,0.0,126.0,204.0,0.0,0.0,50.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,11,0,0,0,0,1,0


In [22]:
y_lasso_predict = lasso.predict(yy)
submission['num'] = y_lasso_predict

submission.to_csv('lasso.csv', index=False)